# INSTALACION DE LIBRERIAS

In [ ]:
!pip install bitsandbytes accelerate xformers einops langchain faiss-cpu transformers pypdf sentence-transformers -q

# IMPORTS E INICIALIZACION

In [ ]:
from typing import List
import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.chains import ConversationalRetrievalChain, RetrievalQA
from langchain.callbacks.tracers import ConsoleCallbackHandler
from langchain_core.vectorstores import VectorStoreRetriever
from langchain.memory import ConversationBufferMemory
from langchain.vectorstores import FAISS

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)
if device == 'cuda':
    print(torch.cuda.get_device_name(0))


Device: cpu


In [ ]:
orig_model_path = "mistralai/Mistral-7B-Instruct-v0.1"
model_path = "filipealmeida/Mistral-7B-Instruct-v0.1-sharded"
bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16,
                               )
if device == 'cuda':
  model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, quantization_config=bnb_config, device_map="auto") #esto parece que solo anda con gpu t4
else
  model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True, device_map="auto",offload_folder="/content/extra") #cpu (se queda sin ram colab y se rompe)
tokenizer = AutoTokenizer.from_pretrained(orig_model_path)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
text_generation_pipeline = transformers.pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
    repetition_penalty=1.1,
    return_full_text=True,
    max_new_tokens=100,
)
mistral_llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

# PRUEBAS
se pueden omitir, las hice para ver como funcionan algunas cosas

In [ ]:
text = "que es mistral? escribe una respuesta corta."
mistral_llm.invoke(text)

#> Mistral is a type of cold front that forms over the Mediterranean
#> Sea and moves eastward across southern Europe, bringing strong winds
#> and sometimes severe weather conditions such as heavy rainfall, hail,
#> and even tornadoes.

In [ ]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "dime un chiste {adjective} sobre {content}."
)
prompt.format(adjective="divertido", content="pollos")

llm_chain = prompt | mistral_llm
llm_chain.invoke({"adjective": "divertido", "content": "pollos"})

In [ ]:
llm_chain = prompt | mistral_llm
llm_chain.invoke({"adjective": "divertido", "content": "pollos"},
                 config={'callbacks': [ConsoleCallbackHandler()]})

# AGREGO PDFS Y GUARDO EN DB VECTORIAL

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from langchain.document_loaders import PyPDFDirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

# carga de PDFs del directorio
loader = PyPDFDirectoryLoader("/content/drive/MyDrive/milei/")
data = loader.load()

# Text chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)
text_chunks = text_splitter.split_documents(data)

# preparacion de textos, metadata e ids
documents = []
metadata = []
ids = []

#print(text_chunks[2].chunk)
for i, chunk in enumerate(text_chunks):
  documents.append(chunk.page_content)
  metadata.append(data[i].metadata)
  ids.append(str(i))

# embeddings

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-l6-v2",
    model_kwargs={"device": "cuda"},
)

vector_db = FAISS.from_texts(texts=documents, embedding=embeddings, metadatas=metadata, ids=ids)
retriever = VectorStoreRetriever(vectorstore=vector_db, k=3)

In [ ]:
template = """eres una IA asistente muy servicial. usa las siguientes partes de contexto para responder al final.
              {context}
              historial de chat: {history}
              pregunta: {question}
              respuesta:"""

prompt = PromptTemplate(
        template=template, input_variables=["history", "context", "question"],
        hide_template=True,  # Oculta el template del log
    )
qa = RetrievalQA.from_chain_type(
        llm=mistral_llm,
        chain_type="stuff",
        retriever=retriever)
#        chain_type_kwargs={
#            "verbose": True,
#            "prompt": prompt,
#            "memory": ConversationBufferMemory(
#                memory_key="history",
#                input_key="question"),
#        }
#    )
# deje comentado la agregacion del template a la definicion de qa

In [ ]:
qa.invoke("nombrame un articulo de la constitucion nacional")

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa.invoke({'query': user_input})
  print(f"Answer: {result['result']}")

#me responde en ingles por mas que la pregunta y la documentacion estan en español